In [2]:
import pandas as pd
import numpy as np
import os as os
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
!pip install awswrangler

import awswrangler as wr

  Obtaining dependency information for awswrangler from https://files.pythonhosted.org/packages/54/fc/8d618376fecdc0e2f6d88d1c04f10a2d9d374bfbc14476b0f88d93dc697d/awswrangler-3.4.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 931.2 kB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for urllib3<1.27,>=1.25.4 from https://files.pythonhosted.org/packages/c5/05/c214b32d21c0b465506f95c4f28ccbcba15022e000b043b72b3df7728471/urllib3-1.26.16-py2.py3-none-any.whl.metadata
  Using cached urllib3-1.26.16-py2.py3-none-any.whl.metadata (48 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.0/395.0 kB 6.1 MB/s eta 0:00:00:00:01
Using cached urllib3-1.26.16-py2.py3-none-any.whl (143 kB)
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version

In [6]:
from sqlalchemy import create_engine
import os
import boto3

### Obtenção de dados - fonte da tabela na pasta sql

In [4]:
query = """
  
with gestao_carteira as (
select
cpf
, max(vl_current_limit) as max_limite
, sum(vl_util_will) as sum_utilizado_interno
, sum(vl_util_market) as sum_utilizado_mercado
from customer_curated_zone.ca_book_gestao_carteira 
group by 1
)
, cobranca as (
select
id_customer
, max(nr_days_paste_due_current) as nr_days_paste_due_current
from (
	select
	id_customer , nr_days_paste_due_current 
	, row_number() over (partition by id_customer order by cd_yearmonth desc) as num_ordem
	from customer_curated_zone.ca_book_cobranca 
)
where num_ordem = 1
group by 1
)
, features_conta as (
	select 
	cpf 
	, max(nr_transacoes_cashin + nr_transacoes_pix_cashout + nr_transacoes_ted_cashout) as nr_trans_mov_conta
	from customer_analytics_zone.ca_base_calculo_ie_conta
	group by 1
)
, eventos as (
	select 
	cpf
	, max(case when lower(ds_event_name) like '%click_button_sendinvite_telamgm%' or lower(ds_event_name) like '%click_card_mgm_timeline%' or lower(ds_event_name) like '%click_button_sendinvite_screenmgmhome%' then 1 else 0 end) as flag_mgm
	from customer_curated_zone.ca_analitico_frontend
	where lower(ds_event_name) <> 'login_knownuser'
	and lower(ds_event_name) <> 'insert_textfield_password_loginknownuser'
	and lower(ds_event_name) <> 'home'
	and lower(ds_event_name) <> 'click_box_tracking_card_home'
	group by 1
)
, base_principal as (
 select 
	cli2.cpf 
	, max(date_diff('day', cast(pr.dt_cfi_account as date), current_date)) as tempo_relacionamento
	, max(gc.max_limite) - min(pr.nr_approved_limit_engine) as ganho_limite
	, sum(gc.sum_utilizado_interno) as sum_utilizado_interno
	, sum(gc.sum_utilizado_mercado) as sum_utilizado_mercado
	, sum(ic.nr_trans_mov_conta) as nr_trans_mov_conta
	, max(e.flag_mgm) as flag_mgm
	, max(cob.nr_days_paste_due_current) as dias_atraso
from customer_curated_zone.ca_book_status_cliente cli1
	inner join (
		select distinct id_customer, cpf from 
		customer_curated_zone.ca_book_cliente) cli2 
		on (cli1.id_customer = cli2.id_customer)
	left join growth_curated_zone.proposal_analysis pr on (pr.cpf = cli2.cpf)
	left join gestao_carteira gc on (gc.cpf = cli2.cpf)
	left join features_conta ic on (ic.cpf = cli2.cpf)
	left join cobranca cob on (cob.id_customer = cli1.id_customer)
	left join eventos e on (e.cpf = cli2.cpf)
where pr.ds_approval_type = 'credito' and cob.nr_days_paste_due_current <= 5
group by 1
)
, base_final as (
select 
*
, coalesce(coalesce(sum_utilizado_interno, 0.0)/(coalesce(sum_utilizado_interno, 0.0) + coalesce(sum_utilizado_mercado, 0.0)),0) as share_of_wallet
from base_principal
)
select * from base_final

"""

In [7]:
staging_path = 's3://will-prod-ml-platform-sagemaker-studio/staging/flavia-costa/'
boto3_session = boto3.Session(region_name='sa-east-1')

# Clean temporary folder
wr.s3.delete_objects(staging_path, boto3_session=boto3_session)

# Run query
print('Started query')

df = wr.athena.read_sql_query(
    sql=query,
    database=None,
    unload_approach=True,
    ctas_approach=False,
    boto3_session=boto3_session,
    s3_output=staging_path
)
wr.s3.delete_objects(staging_path, boto3_session=boto3_session)

Started query


In [8]:
len(df)

2806610

In [9]:
df.dtypes

cpf                       string
tempo_relacionamento       Int64
ganho_limite             float64
sum_utilizado_interno     object
sum_utilizado_mercado    float64
nr_trans_mov_conta       float64
flag_mgm                   Int32
dias_atraso                Int32
share_of_wallet          float64
dtype: object

In [10]:
df.isnull().sum()

cpf                            0
tempo_relacionamento       10449
ganho_limite              254336
sum_utilizado_interno     311628
sum_utilizado_mercado     311628
nr_trans_mov_conta       1017908
flag_mgm                  986137
dias_atraso                    0
share_of_wallet           339632
dtype: int64

### Definindo tipos de var e lidando com missings

In [15]:
#variáveis para recodificar como zero
df['share_of_wallet'] = df['share_of_wallet'].fillna(0.0)
df['ganho_limite'] = df['ganho_limite'].fillna(0.0)
df['flag_mgm'] = df['flag_mgm'].fillna(0.0)
df['nr_trans_mov_conta'] = df['nr_trans_mov_conta'].fillna(0.0)


In [16]:
df.isnull().sum()

cpf                           0
tempo_relacionamento      10449
ganho_limite                  0
sum_utilizado_interno    311628
sum_utilizado_mercado    311628
nr_trans_mov_conta            0
flag_mgm                      0
dias_atraso                   0
share_of_wallet               0
dtype: int64

In [42]:
len(df.query('ganho_limite < 0'))

8728

In [43]:
df.query('ganho_limite < 0')['flag_mgm'].mean()

0.012144821264894592

In [17]:
for column in df.columns:
  if df[column].dtype == 'object':
    df[column].fillna(df[column].mode()[0], inplace = True)
  elif df[column].dtype == 'string':
    df[column].fillna(df[column].mode()[0], inplace = True)
  elif df[column].dtype != 'object':
    df[column].fillna(df[column].median(), inplace = True)

In [18]:
df.isnull().sum()

cpf                      0
tempo_relacionamento     0
ganho_limite             0
sum_utilizado_interno    0
sum_utilizado_mercado    0
nr_trans_mov_conta       0
flag_mgm                 0
dias_atraso              0
share_of_wallet          0
dtype: int64

In [19]:
df['flag_mgm'].value_counts()

0    2770511
1      36099
Name: flag_mgm, dtype: Int64

In [1]:
36099 / (36099 + 2770511)

0.012862136171395384

## Início modelagem

In [20]:
def discretiza(df, drop_col):
    num_columns = df.drop(drop_col, axis = 1).select_dtypes(exclude=['object']).columns
    stats = df.drop(drop_col, axis=1).describe()
    for i in num_columns:
        p75 = stats.iloc[6][i]
        p25 = stats.iloc[4][i]
        p50 = stats.iloc[5][i]
        
        df['aux'] = np.where((df[i] <= p25), "A - " + i,
                    np.where((df[i] > p25) & (df[i] <= p50), "B - " + i,
                    np.where((df[i] > p50) & (df[i] <= p75), "C - " + i, "D - " + i)))
  
        df = df.rename(columns = {'aux':i+"_fx"})

    return df

In [21]:
df = discretiza(df, ['cpf', 'flag_mgm'])

In [22]:
df.columns

Index(['cpf', 'tempo_relacionamento', 'ganho_limite', 'sum_utilizado_interno',
       'sum_utilizado_mercado', 'nr_trans_mov_conta', 'flag_mgm',
       'dias_atraso', 'share_of_wallet', 'tempo_relacionamento_fx',
       'ganho_limite_fx', 'sum_utilizado_mercado_fx', 'nr_trans_mov_conta_fx',
       'dias_atraso_fx', 'share_of_wallet_fx'],
      dtype='object')

In [ ]:
'''
seg_relacionamento
seg_mov_conta
seg_share_of_wallet
seg_ganho_limite
segmento_v2 = df.seg_ganho_limite.astype("string") + df.seg_relacionamento.astype("string") + df.seg_mov_conta.astype("string")

'''

In [35]:
df['seg_relacionamento'] = np.where(df.tempo_relacionamento <=  df.tempo_relacionamento.quantile(0.25), 'A',
                                  np.where(df.tempo_relacionamento <=  df.tempo_relacionamento.quantile(0.75), 'B', 'C'))

df['seg_mov_conta'] = np.where(df.nr_trans_mov_conta <=  df.nr_trans_mov_conta.quantile(0.75), 'A', 'B')

df['seg_share_of_wallet'] = np.where(df.share_of_wallet <=  df.share_of_wallet.quantile(0.50), 'A', 'B')

df['seg_ganho_limite'] = np.where(df.ganho_limite <=  df.ganho_limite.quantile(0.25), 'A',
                                  np.where(df.ganho_limite <=  df.ganho_limite.quantile(0.75), 'B', 'C'))

In [36]:
df['segmento_v2'] = df.seg_ganho_limite.astype("string") + df.seg_relacionamento.astype("string") + df.seg_mov_conta.astype("string")

In [37]:
df['segmento_final'] = np.where(df.seg_relacionamento == "A", "0. Pouco tempo de relacionamento",
np.where(df.seg_relacionamento.astype("string") + df.seg_mov_conta.astype("string") == "CA", "0. Alto Tempo de Relac - Pouco Movimento Conta",
#Abrir por share of wallet
np.where((df.segmento_v2 == "ABA") & (df.seg_share_of_wallet == "A"), "1. Tempo de Relac Mediano - Pouco aumento limite - Pouco Mov Conta - Baixo SOW", 
np.where((df.segmento_v2 == "ABA") & (df.seg_share_of_wallet == "B"), "2. Tempo de Relac Mediano - Pouco aumento limite - Pouco Mov Conta - Alto SOW",
np.where(df.segmento_v2 == "ABB", "3. Tempo de Relac Mediano - Pouco aumento limite - Alto Movimento Conta",
np.where(df.segmento_v2 == "BBB", "5. Tempo de Relac Mediano - Aumento limite mediano - Alto Movimento Conta",
np.where(df.segmento_v2 == "ACB", "1. Alto Tempo de Relac - Pouco aumento limite - Alto Movimento Conta",
np.where(df.segmento_v2 == "CCB", "1. Alto Tempo de Relac - Alto aumento de limite - Alto Movimento Conta",
np.where(df.segmento_v2 == "BCB", "0. Alto Tempo de Relac - Aumento limite mediano - Alto Movimento Conta",
np.where(df.segmento_v2 == "BBA", "3. Tempo de Relac Mediano - Aumento limite mediano - Pouco Movimento Conta",
np.where(df.segmento_v2 == "CBA", "2. Tempo de Relac Mediano - Aumento limite mediano - Baixo Movimento Conta",
np.where(df.segmento_v2 == "CBB", "4. Tempo de Relac Mediano - Alto aumento limite - Alto Movimento Conta", "NI"))))))))))))

df[['segmento_final', 'flag_mgm']].groupby('segmento_final').agg({'flag_mgm':['mean', 'count']})

flag_mgm        
                                                        mean   count
segmento_final                                                      
0. Alto Tempo de Relac - Aumento limite mediano...  0.003716   12916
0. Alto Tempo de Relac - Pouco Movimento Conta      0.001979  545747
0. Pouco tempo de relacionamento                    0.000004  703255
1. Alto Tempo de Relac - Alto aumento de limite...  0.006193   87518
1. Alto Tempo de Relac - Pouco aumento limite -...  0.004793   55080
1. Tempo de Relac Mediano - Pouco aumento limit...  0.008267  250159
2. Tempo de Relac Mediano - Aumento limite medi...  0.021884  256033
2. Tempo de Relac Mediano - Pouco aumento limit...   0.01527  141522
3. Tempo de Relac Mediano - Aumento limite medi...  0.025361  288044
3. Tempo de Relac Mediano - Pouco aumento limit...  0.027664  131544
4. Tempo de Relac Mediano - Alto aumento limite...  0.033497  140790
5. Tempo de Relac Mediano - Aumento limite medi...   0.04469  194002

In [33]:
df['flag_mgm'].mean()

0.012862136171395384

In [38]:
df['meses_relacionamento'] = df['tempo_relacionamento'] / 30

df[['segmento_final', 'meses_relacionamento']].groupby('segmento_final').agg({'meses_relacionamento':['mean', 'min', 'max']})

meses_relacionamento  \
                                                                   mean   
segmento_final                                                            
0. Alto Tempo de Relac - Aumento limite mediano...            45.846735   
0. Alto Tempo de Relac - Pouco Movimento Conta                53.553431   
0. Pouco tempo de relacionamento                               2.305037   
1. Alto Tempo de Relac - Alto aumento de limite...            47.414336   
1. Alto Tempo de Relac - Pouco aumento limite -...            61.144575   
1. Tempo de Relac Mediano - Pouco aumento limit...            15.234962   
2. Tempo de Relac Mediano - Aumento limite medi...            22.045097   
2. Tempo de Relac Mediano - Pouco aumento limit...            13.406059   
3. Tempo de Relac Mediano - Aumento limite medi...            15.314185   
3. Tempo de Relac Mediano - Pouco aumento limit...            14.167389   
4. Tempo de Relac Mediano - Alto aumento limite...            22.340228   
5. Tempo de Relac Mediano - Aumento limite medi...            15.727682   

                                                                          
                                                          min        max  
segmento_final                                                            
0. Alto Tempo de Relac - Aumento limite mediano...  35.466667       77.0  
0. Alto Tempo de Relac - Pouco Movimento Conta      35.466667  77.033333  
0. Pouco tempo de relacionamento                     0.033333        4.9  
1. Alto Tempo de Relac - Alto aumento de limite...  35.466667       77.0  
1. Alto Tempo de Relac - Pouco aumento limite -...  35.466667       77.0  
1. Tempo de Relac Mediano - Pouco aumento limit...   4.933333  35.366667  
2. Tempo de Relac Mediano - Aumento limite medi...   4.933333  35.366667  
2. Tempo de Relac Mediano - Pouco aumento limit...   4.933333  35.366667  
3. Tempo de Relac Mediano - Aumento limite medi...   4.933333  35.366667  
3. Tempo de Relac Mediano - Pouco aumento limit...   4.933333  35.366667  
4. Tempo de Relac Mediano - Alto aumento limite...   4.933333  35.366667  
5. Tempo de Relac Mediano - Aumento limite medi...   4.933333  35.366667

In [39]:
df[['segmento_final', 'ganho_limite']].groupby('segmento_final').agg({'ganho_limite':['mean', 'min', 'max']})

ganho_limite          \
                                                           mean     min   
segmento_final                                                            
0. Alto Tempo de Relac - Aumento limite mediano...   415.426603    10.0   
0. Alto Tempo de Relac - Pouco Movimento Conta      2092.182015 -7400.0   
0. Pouco tempo de relacionamento                      20.574066 -9100.0   
1. Alto Tempo de Relac - Alto aumento de limite...  5318.948182   744.0   
1. Alto Tempo de Relac - Pouco aumento limite -...    -0.276870 -3000.0   
1. Tempo de Relac Mediano - Pouco aumento limit...   -19.587259 -9900.0   
2. Tempo de Relac Mediano - Aumento limite medi...  2456.784161   743.0   
2. Tempo de Relac Mediano - Pouco aumento limit...    -2.565749 -3700.0   
3. Tempo de Relac Mediano - Aumento limite medi...   362.237318    10.0   
3. Tempo de Relac Mediano - Pouco aumento limit...    -7.745849 -9800.0   
4. Tempo de Relac Mediano - Alto aumento limite...  2178.905284   743.0   
5. Tempo de Relac Mediano - Aumento limite medi...   355.059597    10.0   

                                                              
                                                         max  
segmento_final                                                
0. Alto Tempo de Relac - Aumento limite mediano...     740.0  
0. Alto Tempo de Relac - Pouco Movimento Conta       98300.0  
0. Pouco tempo de relacionamento                     20200.0  
1. Alto Tempo de Relac - Alto aumento de limite...   29055.0  
1. Alto Tempo de Relac - Pouco aumento limite -...       0.0  
1. Tempo de Relac Mediano - Pouco aumento limit...       0.0  
2. Tempo de Relac Mediano - Aumento limite medi...  199000.0  
2. Tempo de Relac Mediano - Pouco aumento limit...       0.0  
3. Tempo de Relac Mediano - Aumento limite medi...     740.0  
3. Tempo de Relac Mediano - Pouco aumento limit...       0.0  
4. Tempo de Relac Mediano - Alto aumento limite...   67460.0  
5. Tempo de Relac Mediano - Aumento limite medi...     740.0

In [40]:
df[['segmento_final', 'nr_trans_mov_conta']].groupby('segmento_final').agg({'nr_trans_mov_conta':['mean', 'min', 'max']})

nr_trans_mov_conta        \
                                                                 mean   min   
segmento_final                                                                
0. Alto Tempo de Relac - Aumento limite mediano...          91.581527  17.0   
0. Alto Tempo de Relac - Pouco Movimento Conta               2.732010   0.0   
0. Pouco tempo de relacionamento                             7.457326   0.0   
1. Alto Tempo de Relac - Alto aumento de limite...          87.705809  17.0   
1. Alto Tempo de Relac - Pouco aumento limite -...          82.430247  17.0   
1. Tempo de Relac Mediano - Pouco aumento limit...           2.597240   0.0   
2. Tempo de Relac Mediano - Aumento limite medi...           4.914925   0.0   
2. Tempo de Relac Mediano - Pouco aumento limit...           5.994156   0.0   
3. Tempo de Relac Mediano - Aumento limite medi...           5.825155   0.0   
3. Tempo de Relac Mediano - Pouco aumento limit...          76.835857  17.0   
4. Tempo de Relac Mediano - Alto aumento limite...          82.747965  17.0   
5. Tempo de Relac Mediano - Aumento limite medi...          78.954408  17.0   

                                                             
                                                        max  
segmento_final                                               
0. Alto Tempo de Relac - Aumento limite mediano...   8472.0  
0. Alto Tempo de Relac - Pouco Movimento Conta         16.0  
0. Pouco tempo de relacionamento                     6027.0  
1. Alto Tempo de Relac - Alto aumento de limite...  22100.0  
1. Alto Tempo de Relac - Pouco aumento limite -...  20913.0  
1. Tempo de Relac Mediano - Pouco aumento limit...     16.0  
2. Tempo de Relac Mediano - Aumento limite medi...     16.0  
2. Tempo de Relac Mediano - Pouco aumento limit...     16.0  
3. Tempo de Relac Mediano - Aumento limite medi...     16.0  
3. Tempo de Relac Mediano - Pouco aumento limit...  39064.0  
4. Tempo de Relac Mediano - Alto aumento limite...  18315.0  
5. Tempo de Relac Mediano - Aumento limite medi...   9339.0

In [41]:
df[['segmento_final', 'share_of_wallet']].groupby('segmento_final').agg({'share_of_wallet':['mean', 'min', 'max']})

share_of_wallet            \
                                                              mean       min   
segmento_final                                                                 
0. Alto Tempo de Relac - Aumento limite mediano...        0.420386  0.000000   
0. Alto Tempo de Relac - Pouco Movimento Conta            0.295107  0.000000   
0. Pouco tempo de relacionamento                          0.149574  0.000000   
1. Alto Tempo de Relac - Alto aumento de limite...        0.504439  0.000000   
1. Alto Tempo de Relac - Pouco aumento limite -...        0.501518  0.000000   
1. Tempo de Relac Mediano - Pouco aumento limit...        0.055378  0.000000   
2. Tempo de Relac Mediano - Aumento limite medi...        0.326148  0.000000   
2. Tempo de Relac Mediano - Pouco aumento limit...        0.457295  0.237432   
3. Tempo de Relac Mediano - Aumento limite medi...        0.306406  0.000000   
3. Tempo de Relac Mediano - Pouco aumento limit...        0.373499  0.000000   
4. Tempo de Relac Mediano - Alto aumento limite...        0.413991  0.000000   
5. Tempo de Relac Mediano - Aumento limite medi...        0.410588  0.000000   

                                                              
                                                         max  
segmento_final                                                
0. Alto Tempo de Relac - Aumento limite mediano...  1.000000  
0. Alto Tempo de Relac - Pouco Movimento Conta      1.000000  
0. Pouco tempo de relacionamento                    1.000000  
1. Alto Tempo de Relac - Alto aumento de limite...  1.000000  
1. Alto Tempo de Relac - Pouco aumento limite -...  1.000000  
1. Tempo de Relac Mediano - Pouco aumento limit...  0.237432  
2. Tempo de Relac Mediano - Aumento limite medi...  1.000000  
2. Tempo de Relac Mediano - Pouco aumento limit...  1.000000  
3. Tempo de Relac Mediano - Aumento limite medi...  1.000000  
3. Tempo de Relac Mediano - Pouco aumento limit...  1.000000  
4. Tempo de Relac Mediano - Alto aumento limite...  1.000000  
5. Tempo de Relac Mediano - Aumento limite medi...  1.000000

In [ ]:
'''
PROXIMOS PASSOS:

- CRUZAR COM BASE DE PESQUISA
- POLIS

'''



In [44]:
wr.s3.to_csv(
    df=df,
    path='s3://data-athena-query-result-will-prod/customer.segmentacao_mgm.csv',
    dataset=True, 
    mode="overwrite",
    database='customer_sandbox_zone',  
    table='segmentacao_mgm'  
)

{'paths': ['s3://data-athena-query-result-will-prod/customer.segmentacao_mgm.csv/9a116e945a7d453595f2544528791937.csv'],
 'partitions_values': {}}